In [25]:
class Dealer():
    def __init__(self, deck):
        # initialize empty hand
        self.hand = []
        self.ace = 0
        self.above_21 = False
        # get 2 card by .hit_card() method of deck
        #verify aces, reduce value if aove 21 and have aces
        for n in range(0,2):
            self.card = deck.hit_card()
            self.hand.append(self.card)
            if self.card.rank == 'Ace':
                self.ace += 1
        self.value = self.update_value()
        
    # moves to do: hit or stand
    def hit(self, deck):
        self.card = deck.hit_card()
        print('Card drawn: ')
        print(self.card.suite, self.card.rank, self.card.value)
        self.hand.append(self.card)
        self.value = self.update_value()

    def update_value(self):
        self.value = sum([card.value for card in self.hand])
        if self.value > 21 and self.ace >0:
            self.value -=10
        elif self.value >21 and self.ace <= 0:
            self.above_21 = True
        return self.value

    # one card of dealers hand is visible
    def show_dealer(self):
        card = self.hand[0]
        print('dealers got: ', card.suite, card.rank, card.value)

    # For development - show hand and value etc
    def show_hand(self):
        for card in self.hand:
            print(card.suite, card.rank, card.value)
        print('hand value: ', self.value)
        print('have ace ?: ', self.ace)

In [26]:
class Player(Dealer):
       def __init__(self, deck, money, name):
                super().__init__(deck)
       # Player inherit from Dealer but need additional attribute money
                self.money = money
                self.name = name
                self.queue = 'initialization'
                self.next = None
       # function for queue management
       def set_queue(self, value):
               if value in ["Active","Waiting","Standing","Bust","BlackJack"]:
                      self.queue = value
               else:
                     print('unknown atribute')
       def set_next(self, player):
               self.next = player

       # money management
       def win_money(self, ammount):
               self.money += ammount
       # For development - check money
       def check_money(self):
            print('You ve got, ', self.money)


In [27]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

Card object - need to store cards attributes

In [28]:
class Card():
    def __init__(self, suite, rank):
        self.suite= suite
        self.rank= rank
        self.value= values[rank]

Deck object - need to store attributes and methods - shuffle / hit card

In [29]:
import random
class Deck():
    def __init__(self):
        self.set_of_all=[]
        for suite in suits:
            for rank in ranks:
                self.set_of_all.append(Card(suite, rank))
        #always shuffle new set
        self.shuffle()
    def shuffle(self):
        print('deck shuffling')
        print(len(self.set_of_all), ' cards in deck')
        random.shuffle(self.set_of_all)
    def hit_card(self):
        return self.set_of_all.pop()
    def __str__(self):
        return str(len(self.set_of_all))

Game queue - for queue management

In [30]:
class Game_queue():
  def __init__(self):
    self.list_of_players=[]
    self.is_active = False

  # add new player and set queue
  # if new player is added -> he becomes next for previous player
  # AND player 0 must become next for last in list (so it goes in circle)
  def add_player(self,player):
    player.set_queue("Waiting")
    self.list_of_players.append(player)
    if self.is_active == False:
      player.set_queue("Active")
      self.is_active = True
    if len(self.list_of_players) <= 1:
      player.set_next(player)
    elif len(self.list_of_players) > 1:
      self.list_of_players[-2].set_next(player)
      self.list_of_players[-1].set_next(self.list_of_players[0])

  def check_queue(self):
    for player in self.list_of_players:
      print(player.name, player.queue, " next - " , player.next.name)
  # set player "Standing, verify if there is someone waiting. If None waiting, queue is finished
  # need to create method for new round after dealers move and bet checking
  def player_stand(self):
    for player in self.list_of_players:
      if player.queue == 'Active':
        player.set_queue("Standing")
        queue_status = set([player.queue for player in self.list_of_players])
        queue_status = list(queue_status)
        if (queue_status == ["Standing"]) == True:
          break
        else:
          player.next.set_queue("Active")
          break
  def whos_turn(self):
    for player in self.list_of_players:
      if player.queue == "Active":
        return player
    return False
  def list_of_players(self):
    return self.list_of_players

In [31]:
q = Game_queue()
deck = Deck()
player1 = Player(deck, 100, 'Player1')
player2 = Player(deck, 100, 'Player2')
player3 = Player(deck, 100, 'Player3')
player4 = Player(deck, 100, 'Player4')

q.add_player(player1)
q.add_player(player2)
q.add_player(player3)
q.add_player(player4)

deck shuffling
52  cards in deck


Winner checker

In [32]:
class Winner_checker():
    def __init__(self):
        self.players_ranking = {}

# it is starting to make sense - creating ex equo ranks
    def rank_dealer(self, game):
        self.over_21, self.under_21 = game.value_of_players()
        self.players_ranking[self.under_21[0]] = 1
        for position in range(1,len(self.under_21)):

            if self.under_21[position].value == self.under_21[position-1].value:
                self.players_ranking[self.under_21[position]] = self.players_ranking[self.under_21[position-1]]
            else:
                self.players_ranking[self.under_21[position]] = self.players_ranking[self.under_21[position-1]]+1
        for element in wc.players_ranking.items():
            print()
            try:
                print(element[0].name)
            except:
                print('dealer?')
            print(' position ',element[1], ' value ',element[0].value, ' hand len ', len(element[0].hand))

    def instant_loose(self):
        """
        Check if dealer alone in under_21 group.
        This gives instant loose
        """
        if (dealer in self.under_21) and (len(self.under_21) == 1):
            return True
        if (dealer in self.under_21) and (len(self.under_21) != 1):
            for player in self.under_21:
                if player != dealer and player.value > self.under_21[self.under_21.index(dealer)].value:    #value of dealer
                    return False
            return True
        
    def instant_tie(self):
        """
        Check if any player have same result as dealer.
        Returns list of player who tie with dealer
        """
        self.is_tie = False
        self.list_of_ties = []
        if (dealer in self.under_21) and (len(self.under_21) != 1):
            for player in self.under_21:
                if player != dealer and player.value == self.under_21[self.under_21.index(dealer)].value:    #value of dealer
                    self.is_tie = True
                    self.list_of_ties.append(player)
        return self.list_of_ties
        
    def find_dealer(self):
        """
        Check if dealer is under or over 21 pts.
        Check if dealer is alone or not in group.
        Check position of dealer in group
        """
        if dealer in self.under_21:
            return self.under_21, len(self.under_21) ,self.under_21.index(dealer)
        else:
            return self.over_21, len(self.over_21), self.over_21.index(dealer)
    
    
    def is_black_jack(self):
        pass


Game engine

In [33]:
import time
from IPython.display import clear_output
class Game_engine():

    def __init__(self, dealer, Game_queue, deck):
        print('Game starts')
        self.dealer = dealer
        self.deck = deck
        self.bet = []
        self.Game_queue = Game_queue
        self.input=''

    def game_turn(self):
        self.player = self.Game_queue.whos_turn()

        if self.player != False:
            self.input = input(f'{self.player.name} Turn: h or s or hand')
            print('you choose', self.input)
        # HIT!
            if self.input == 'h':
                self.player.hit(self.deck)
                print(self.player.name, ' value: ', self.player.value)
                if self.player.value >21:
                    print(self.player,' loose')
                    self.Game_queue.player_stand()
        # STAND!
            elif self.input == 's':
                print(self.player.name ,'value: ', self.player.value)
                #clear_output(wait=True)
                self.Game_queue.player_stand()
        # CHECK HAND
            elif self.input == 'hand':
                self.player.show_hand()
                self.dealer.show_dealer()
            self.game_turn()
        
        elif self.input == 'e':
                self.exit()    
            
# DEALERS TURN if below 17 - hit, over 17 stand
        elif self.player == False:
            print(self.dealer.show_hand())
            if self.dealer.value < 17:
                self.dealer.hit(self.deck)
                print('dealr hits',  self.dealer.value)
                self.game_turn()
            elif self.dealer.value >= 17:
                print('dealer stand',  self.dealer.value)
                wc.rank_dealer(self)
                
#return sorted dict of players / values sorted by value
    def value_of_players(self):
        self.players_values = {}
        self.list_of_players = self.Game_queue.list_of_players
        self.players_values[self.dealer] = self.dealer.value
        for player in self.list_of_players:
            self.players_values[player] = player.value
        
        
        # added second level for test - key= lambda x:len(x[0].hand)
        condition = lambda x: (x[1] , -len(x[0].hand))
        self.players_values = sorted(self.players_values.items(), key=condition, reverse=True)
        self.players_values = dict(self.players_values)
        self.over_21 = [player for player in self.players_values if player.value > 21]
        self.eq_under_21 = [player for player in self.players_values if player.value <= 21]
        return self.over_21, self.eq_under_21

    def check_bet(self):
# WIN/LOOSE CONDITION
       print('checking who won')
       print('so far i`m finishing :)')
       self.exit()
# @@@@@@@@@@@@@@@ TBU @@@@@@@@@@@@@@@
# scenario if player loose
    def player_loose(self):
        print('player lost')
        print('reinitialising player dealer and bet')
        self.bet = []
        self.player.__init__(self.deck, self.player.money)
        self.dealer.__init__(self.deck)

    def player_win(self):
        print('player wins')
        self.player.win_money(sum(self.bet))
        print('reinitialising player dealer and bet')
        self.bet = []
        self.player.__init__(self.deck, self.player.money)
        self.dealer.__init__(self.deck)

# @@@@@@@@@@@@@@@ TBU @@@@@@@@@@@@@@@


    def exit(self):
        print('exit')

In [34]:
q = Game_queue()
deck = Deck()
player1 = Player(deck, 100, 'Player1')
player2 = Player(deck, 100, 'Player2')
player3 = Player(deck, 100, 'Player3')
player4 = Player(deck, 100, 'Player4')

q.add_player(player1)
q.add_player(player2)
q.add_player(player3)
q.add_player(player4)

deck = Deck()
dealer = Dealer(deck)
game = Game_engine( Game_queue=q, dealer=dealer, deck=deck)
wc = Winner_checker()



deck shuffling
52  cards in deck
deck shuffling
52  cards in deck
Game starts


In [35]:
game.game_turn()

you choose hand
Clubs Five 5
Diamonds Two 2
hand value:  7
have ace ?:  0
dealers got:  Hearts King 10
you choose h
Card drawn: 
Hearts Ten 10
Player1  value:  17
you choose s
Player1 value:  17
you choose hand
Hearts Three 3
Hearts Four 4
hand value:  7
have ace ?:  0
dealers got:  Hearts King 10
you choose h
Card drawn: 
Spades Two 2
Player2  value:  9
you choose h
Card drawn: 
Spades Queen 10
Player2  value:  19
you choose s
Player2 value:  19
you choose hand
Hearts King 10
Clubs Nine 9
hand value:  19
have ace ?:  0
dealers got:  Hearts King 10
you choose s
Player3 value:  19
you choose hand
Spades Five 5
Clubs Queen 10
hand value:  15
have ace ?:  0
dealers got:  Hearts King 10
you choose h
Card drawn: 
Spades King 10
Player4  value:  25
<__main__.Player object at 0x00000205B1B86D30>  loose
Hearts King 10
Hearts Eight 8
hand value:  18
have ace ?:  0
None
dealer stand 18

Player3
 position  1  value  19  hand len  2

Player2
 position  1  value  19  hand len  4

dealer?
 position 

In [36]:
wc.rank_dealer(game)


Player3
 position  1  value  19  hand len  2

Player2
 position  1  value  19  hand len  4

dealer?
 position  2  value  18  hand len  2

Player1
 position  3  value  17  hand len  3
